# Run a training script with the Python SDK

You can use the Python SDK for Azure Machine Learning to submit scripts as jobs. By using jobs, you can easily keep track of the input parameters and outputs when training a machine learning model.

## Before you start

You'll need the latest version of the **azure-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [12]:
%pip show azure-identity && azure-ai-ml

Name: azure-identity
Version: 1.17.1
Summary: Microsoft Azure Identity Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/identity/azure-identity
Author: Microsoft Corporation
Author-email: azpysdkhelp@microsoft.com
License: MIT License
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: azure-core, cryptography, msal, msal-extensions, typing-extensions
Required-by: opencensus-ext-azure
zsh:1: command not found: azure-ai-ml
Note: you may need to restart the kernel to use updated packages.
  Using cached azureml_dataset_runtime-1.57.0-py3-none-any.whl.metadata (1.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 7.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.7 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
  Using cached azur

## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [9]:
from azureml.core import Workspace
ws = Workspace.from_config(path="../config.json")

## Create Experiment

* Create an experiment 
* Start logging

Experiment is created in Azure with local compute target

In [10]:
from azureml.core import Experiment
import pandas as pd
# create the experiment
experiment = Experiment(workspace=ws, name= "diabetest-train-predict")

# start logging 
run = experiment.start_logging()

## Create the Experiment Python script to train and score a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [11]:
%%writefile diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values
y = diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))


Overwriting diabetes-training.py


## Submit Script to the run as an experiment

Run the cell below to submit the job that trains a classification model to predict diabetes. 

In [15]:
from azureml.core import ScriptRunConfig, Environment

myenv = Environment("user-managed-env")
myenv.python.user_managed_dependencies = True

script_config = ScriptRunConfig(
    source_directory = ".",
    script = "diabetes-training.py",
    environment = myenv
)

run = experiment.submit(config= script_config)
run.wait_for_completion(show_output=True)
#help(script_config)

RunId: diabetest-train-predict_1726057594_1335ebe2
Web View: https://ml.azure.com/runs/diabetest-train-predict_1726057594_1335ebe2?wsid=/subscriptions/ed463f81-92a5-476c-b6e1-82f1a28d21e2/resourcegroups/AzureML/workspaces/ConnectionFromVSCode&tid=746a21b3-a76e-4d67-a142-eeb97ab5314f

Streaming azureml-logs/70_driver_log.txt

/private/var/folders/hc/75f2v8756lz376jb6_px2_ym0000gn/T/azureml_runs/diabetest-train-predict_1726057594_1335ebe2/azureml-setup/context_manager_injector.py:392: SyntaxWarning: invalid escape sequence '\A'
  """
/private/var/folders/hc/75f2v8756lz376jb6_px2_ym0000gn/T/azureml_runs/diabetest-train-predict_1726057594_1335ebe2/azureml-setup/context_manager_injector.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("[{}] Entering context manager injector.".format(datetime.datetime.utcnow().isoformat()))
[2

{'runId': 'diabetest-train-predict_1726057594_1335ebe2',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2024-09-11T12:26:38.748308Z',
 'endTimeUtc': '2024-09-11T12:27:44.988154Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  '_azureml.ClusterName': 'local',
  'ContentSnapshotId': '9059d64a-c65e-473a-8359-dabad5275b24',
  'azureml.git.repository_uri': 'https://github.com/kennedyopokuasare/Azure_datascience.git',
  'mlflow.source.git.repoURL': 'https://github.com/kennedyopokuasare/Azure_datascience.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '5ed9ddd4135e111eed902881633b5097dfbc3cb7',
  'mlflow.source.git.commit': '5ed9ddd4135e111eed902881633b5097dfbc3cb7',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes-training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'fra

## Complete the experiment

In [16]:
run.complete()